<a href="https://colab.research.google.com/github/daripp/leaf-area-counter/blob/main/Leaf_Area_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive',force_remount=True)
#!ls '' 

Mounted at /content/drive


In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import skimage as skimage
import scipy as scipy
import cv2 as cv2
import pandas as pd
import os
from skimage import filters, io, img_as_ubyte
from skimage.color import rgb2grey, label2rgb
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
from skimage.exposure import match_histograms
from skimage.filters import sobel, laplace, gaussian
from skimage import feature
from scipy.ndimage import morphology
from skimage.morphology import disk, binary_dilation, diamond
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
#*********************************************Put folder name here**************************************************
directn = '/content/drive/MyDrive/VitisDrought/VitisGH_Drought2020/Data/WaterPotential/LeafScans/LeafScans_jpg/Misc.misbagged.leaves'
filenames = os.listdir(directn)
for filename in os.listdir(directn): 
#*******************************************makes new directory called "(directory name here) + cropped"********************************************
    new_dir_name=directn+' counted and masked_'
    if not os.path.exists(new_dir_name):
        os.makedirs(new_dir_name)
    #image is read from directory to system  

    
    camera1=io.imread(directn+'/'+filename)
    image = rgb2grey(camera1)
    img=camera1    
    b,g,r = cv2.split(img)        
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Adjust Dilation and remove small objects values here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
    binary_dilation= disk(5)
    remove_small_objects= 2000
    edges_r=np.array(r, dtype='ubyte')
    edges_b=np.array(b, dtype='ubyte')
    edges_g=np.array(g, dtype='ubyte')
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Adjust threshold values here; values are additive!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    edges_r1=np.where((edges_r < 85) & (edges_r > 20), 255, 0)
    edges_b1=np.where((edges_b < 100) & (edges_b > 30), 255, 0)
    edges_g1=np.where((edges_g < 85) & (edges_g > 20), 255, 0)
#!!!!!!!!!!!Images converted to 0-255 scale!!!!!!!!!!!!!!!!!!!!!!!!    
    edges_r2=img_as_ubyte(edges_r1)
    edges_b2=img_as_ubyte(edges_b1)
    edges_g2=img_as_ubyte(edges_g1)
    # #edges2= feature.canny(edges2, sigma=1.5)
#!!!!!!!!!!!!!!!!!!!!Mask expansion, adjust values above!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
    edges_r2=skimage.morphology.binary_dilation(edges_r2, binary_dilation)
    edges_b2=skimage.morphology.binary_dilation(edges_b2, binary_dilation)
    edges_g2=skimage.morphology.binary_dilation(edges_g2, binary_dilation)
###!!!!!!!!!!!!!!!!!!!!Removes small objects marked by the computer!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
    edges_r3 = skimage.morphology.remove_small_objects(edges_r2, min_size=remove_small_objects)
    edges_b3 = skimage.morphology.remove_small_objects(edges_b2, min_size=remove_small_objects)
    edges_g3 = skimage.morphology.remove_small_objects(edges_g2, min_size=remove_small_objects)
###*************************************************!!!!!!!!!!!!!!!!Combine layers here! You don't have to combine if not helpful, but you have to tell the computer what you want!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    edges_combine=edges_r3+edges_b3+edges_g3
#**!!!!!!!!!!!!!!!Fills in holes after layers are combined!!!!!!!!!!!!!!!!!!!!!    
    edges_rb3=scipy.ndimage.morphology.binary_fill_holes(edges_combine)
#!!!!!!!!!!!Generates label from combined images, background is black; we can modify this code to make the masks black if necessary by changing the output colors below!!!!!!!!!!!!!!!
    label_image = label(edges_combine, background=0)
    image_label_overlay = label2rgb(label_image, image=camera1, bg_label=255, alpha=0.2)

    d=measure.regionprops_table(label_image, image, properties=['label','area','centroid'])
    data = pd.DataFrame(d)
    #data.sort_values('area', inplace=True, ascending=False)
    #label_images=label_image<1
    #label_image[label_images]=255
    print(data)
#edges = clear_border(edges)
#camera=np.array(camera)
#camera=np.array(np.count_nonzero(255))
#print(camera)
#inds_x = np.arange(len(camera))
#inds_y = (4 * inds_x) % len(camera)
#camera[inds_x, inds_y] = 0

#io.imshow(edges)
#io.imshow(edges)
    for j in range(len(d['centroid-0'])):
        print(j)

        position = (d['centroid-1'][j],d['centroid-0'][j])
        cv2.putText(
         image_label_overlay, #numpy array on which text is written
         str(d['label'][j]), #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         5, #font size
         (0, 0, 0, 0),15)

    
      
   
        

##########Switching output from 'gray' to 'binary' will invert the masks, making the background white!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
    plt.figure(dpi=300)
    plt.imshow(edges_r3, cmap='gray')
    #plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_red.png')
    plt.close()

    plt.figure(dpi=300)
    plt.imshow(edges_g3, cmap='gray')
   # plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_green.png')
    plt.close()

    plt.figure(dpi=300)
    plt.imshow(edges_b3, cmap='gray')
   # plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_blue.png')
    plt.close()

    plt.figure(dpi=300)
    plt.imshow(edges_combine, cmap='gray')
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_combined.png')
    plt.close()

    plt.figure(dpi=300)
    plt.imshow(image_label_overlay)
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'combined_image_label_overlay.png')
    data['ID'] = ""
    data['Notes'] = ""
    data['scan'] = filename
    data.to_excel(new_dir_name+'/'+filename.split('.')[0]+'combined_image_labeloverlaydata.xlsx')
    plt.close()


    # io.imsave(new_dir_name+'/'+filename.split('.')[0]+'red.jpeg', edges_r3)
    # io.imsave(new_dir_name+'/'+filename.split('.')[0]+'blue.jpeg', edges_b3)
    # io.imsave(new_dir_name+'/'+filename.split('.')[0]+'green.jpeg', edges_g3)

    # data.to_excel(new_dir_name+'/'+filename.split('.')[0]+'.xlsx')

/usr/local/lib/python3.7/dist-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int64 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)


   label     area  centroid-0  centroid-1
0      1   169684         226         590
1      2    10714           3        2014
2      3  1390092        1119        1715
3      4   826012        1970         674
4      5  1422047        2811        1823
5      6   692932        3033         490
0
1
2
3
4
5


/usr/local/lib/python3.7/dist-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int64 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)


   label     area  centroid-0  centroid-1
0      1  1305689        1883         659
1      2   145416          56         867
2      3  1404732         929        1900
3      4   628282         728         634
4      5  1629077        2662        1835
0
1
2
3
4


/usr/local/lib/python3.7/dist-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int64 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)


   label     area  centroid-0  centroid-1
0      1   137519          58         685
1      2     9251           3        2014
2      3    12626         904           3
3      4   465278         587        1614
4      5   657478         690         646
5      6   603643        1462        2067
6      7  1762182        2101         876
7      8  1004528        2956        1835
0
1
2
3
4
5
6
7


/usr/local/lib/python3.7/dist-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int64 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)


   label     area  centroid-0  centroid-1
0      1   139292          57         678
1      2  1034786        2146        1321
2      3     4776        3158        2582
0
1
2


/usr/local/lib/python3.7/dist-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int64 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)


   label     area  centroid-0  centroid-1
0      1   136438          57         684
1      2     7369           2        2064
2      3    12956         910           3
3      4  1889407         984        1797
4      5   966074        1868         820
5      6  1095098        2782        1964
6      7   617193        2920         819
0
1
2
3
4
5
6


In [ ]:
#Code to append the various excel files generated into a single excel file. COMPLETED 2/20

import os
import pandas as pd
import numpy as np
import ntpath
################################### Put the folder name that has the excel files in it ############################
cwd = os.path.abspath('/content/drive/MyDrive/VitisDrought/VitisGH_Drought2020/Data/WaterPotential/LeafScans/LeafScans_jpg/Misc.misbagged.leaves counted and masked_') 
os.chdir(cwd)
files = os.listdir(cwd) 
df = pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        df = df.append(pd.read_excel(cwd+'/'+file), ignore_index=True) 
df['Area_cm2'] = df['area']/14400  #For the jpg scans of the leaves in the GHDrydown 2020 experiment, 60 pixels equaled 1cm so to convert to cm2 I divided by the total pixel# in a cm2. CHECK THIS RATIO FOR FUTURE PROJECTS

filename=ntpath.basename(cwd)
df.to_excel(filename+'_complete.xlsx')